In [13]:
using LinearAlgebra, JuMP, GLPK
x = zeros(Int32, 3,7)
println(x)
for i=1:3
    for j=1:7
        x[i,j] = 1
    end
end
println(x)
x

Int32[0 0 0 0 0 0 0; 0 0 0 0 0 0 0; 0 0 0 0 0 0 0]
Int32[1 1 1 1 1 1 1; 1 1 1 1 1 1 1; 1 1 1 1 1 1 1]


3×7 Array{Int32,2}:
 1  1  1  1  1  1  1
 1  1  1  1  1  1  1
 1  1  1  1  1  1  1

In [16]:
p=[503 140 203; 675 100 45; 630 105 40; 330 40 295; 105 460 120; 490 55 180; 705 60 400]
p

7×3 Array{Int64,2}:
 503  140  203
 675  100   45
 630  105   40
 330   40  295
 105  460  120
 490   55  180
 705   60  400

In [2]:
using GLPK, JuMP, LinearAlgebra
m = Model(with_optimizer(GLPK.Optimizer))
purchased = [5000, 25000, 40000; 10000, 40000, 50000]
market = [1000, 4000, 8000; 1000, 5000, 8000]
sqftPerLogW = zeros(2,2,2,3)
sqftPerLogW[:,:,:,1] = cat([400 400;200 200], [200 200;100 100], dims=3)
sqftPerLogW[:,:,:,2] = cat([700 700;500 500], [350 350;250 250], dims=3)
sqftPerLogW[:,:,:,3] = cat([900 900;1300 1300], [450 450;650 650], dims=3)
@variable(m, w[1:2, 1:2, 1:2] >= 0)             # w(q,v,t)(log)
@variable(m, x[1:2, 1:3] >= 0)                  # x(t,g)(sqft)
@variable(m, y[1:2, 1:3, 1:3] >= 0)             # y(t,g,g)(sheets)
@variable(m, z[1:2, 1:3] >= 0)                  # z(t,g,g)(sheets)
@constraint(m, w[[1,1,1] + w[1,1,2] <= 200])    # log avalability
@constraint(m, w[[1,2,1] + w[1,2,2] <= 100])   
@constraint(m, w[[2,1,1] + w[2,1,2] <= 300])
@constraint(m, w[[2,2,1] + w[2,2,2] <= 1000])
for i=1:2
    for j=1:3
        @constraint(m, x[i,j] == purchased[i,j])# purchased constraint
        @constraint(m, z[i,j] == market[i,j])   # market constraint
    end
end
@constraint(m, 0.25*sum(z[1:]) + 0.4*sum(z[2:]) <= 4500)#pressing constraint
@constraint(m, 400*sum(w[1,:,1]) + 200*sum(w[2,:,1] + x[1,1] >= 35*y))
@constraint(m, )

ErrorException: syntax: unexpected semicolon in array expression

In [27]:
x = [1:2, 1:2, 1:4]
y = zeros(Int32, 2,2,4)
# print(y)
y[1,2,2] = 3
# A = [1 2; 3 4]
A = zeros(2,2,2)
A[:,:,1] = [1 2; 3 4]
A[:,:,2] = [10 20; 30 40]
z = [1:2, 1:2, 1:4]
b = zeros(2,2,2,3)
b[:,:,:,1] = cat([400 400;200 200], [200 200;100 100], dims=3)
b[:,:,:,2] = cat([700 700;500 500], [350 350;250 250], dims=3)
b[:,:,:,3] = cat([900 900;1300 1300], [450 450;650 650], dims=3)
println((A[:,:,1].*b[1,:,:,1]))
println(b[1,:,:,1])
println(A[:,:,1])
print(cat(A[:,:,1], dims=1))
println(b[1,1,1,1])
println(b[1,1,2,1])
println(b[1,2,1,1])
println(b[1,2,2,1])
println(b[2,1,1,1])
println(b[2,1,2,1])
println(b[2,2,1,1])
println(b[2,2,2,1])

[400.0 400.0; 1200.0 800.0]
[400.0 200.0; 400.0 200.0]
[1.0 2.0; 3.0 4.0]
[1.0 2.0; 3.0 4.0]400.0
200.0
400.0
200.0
200.0
100.0
200.0
100.0
